<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial-IllinoisGRMHD: The conservative to primitive algorithm

## Authors: Zach Etienne & Leo Werneck

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we explain the algorithm used to get the primitive variables out of the conservative ones

### Required and recommended citations:

* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

0. [Step 0](#src_dir): **Source directory creation**
1. [Step 1](#introduction): **Introduction**
1. [Step 2](#driver_conserv_to_prims): **`driver_conserv_to_prims.C`**
    1. [Step 2.a](#adm_to_bssn__enforcing_detgammabar_equal_one): *Converting ADM quantities to BSSN quantities and enforcing $\bar\gamma=1$*
    1. [Step 2.b](#equatorial_symmetry): *Applying equatorial symmetry*
    1. [Step 2.c](#variable_setup): *Setting up the variables needed by `HARM`*
        1. [Step 2.c.i](#variable_setup__bssn): BSSN quantities
        1. [Step 2.c.ii](#variable_setup__prims): Primitives
        1. [Step 2.c.iii](#variable_setup__conservs): Conservatives
        1. [Step 2.c.iv](#variable_setup__lapse_and_psi): Lapse function and conformal factor
        1. [Step 2.c.v](#variable_setup__phys_metric): Physical spatial metric
        1. [Step 2.c.vi](#variable_setup__betadown_and_beta2): $\beta_{i}$ and $\beta^{2} \equiv \beta_{i}\beta^{i}$
        1. [Step 2.c.vii](#variable_setup__adm_4metric): The ADM 4-metric, $g_{\mu\nu}$, and its inverse, $g^{\mu\nu}$
        1. [Step 2.c.viii](#variable_setup__temp_conservs): Temporary storage for current values of the conservative variables
    1. [Step 2.d](#conserv_to_prim__driver): *Determining the primitives variables from the conservatives variables*
    
    1. [Step 2.e](#enforce_limits_on_primitives_and_recompute_conservs): *Enforcing physical limits on primitives and recomputing the conservatives variables*
    1. [Step 2.f](#updating_conservs_and_prims_gfs): *Updating conservative and primitive gridfunctions*
    1. [Step 2.g](#diagnostics_and_debugging_tools): *Diagnostics and debugging tools*
1. [Step 3](#harm_primitives_lowlevel): **`harm_primitives_lowlevel.C`**
1. [Step 4](#harm_primitives_headers): **`harm_primitives_headers.h`**
1. [Step 5](#code_validation): **Code validation**
    1. [Step 5.a](#driver_conserv_to_prims_validation): *`driver_conserv_to_prims.C`*
    1. [Step 5.b](#harm_primitives_lowlevel_validation): *`harm_primitives_lowlevel.C`*
    1. [Step 5.c](#harm_primitives_headers_validation): *`harm_primitives_headers.h`*
1. [Step 6](#latex_pdf_output): **Output this module to $\LaTeX$-formatted PDF file**

<a id='src_dir'></a>

# Step 0: Source directory creation \[Back to [top](#toc)\]
$$\label{src_dir}$$

We will now use the [cmdline_helper.py NRPy+ module](Tutorial-Tutorial-cmdline_helper.ipynb) to create the source directory within the `IllinoisGRMHD` NRPy+ directory, if it does not exist yet.

In [1]:
# Step 0: Creation of the IllinoisGRMHD source directory
# Step 0a: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import sys
sys.path.append("../../")

# Step 0b: Load up cmdline_helper and create the directory
import cmdline_helper as cmd
IGM_src_dir_path = "../src"
cmd.mkdir(IGM_src_dir_path)

# Step 0c: Create the output file path 
outfile_path__driver_conserv_to_prims__C  = IGM_src_dir_path+"/driver_conserv_to_prims.C"
outfile_path__harm_primitives_lowlevel__C = IGM_src_dir_path+"/harm_primitives_lowlevel.C"
outfile_path__harm_primitives_headers__h  = IGM_src_dir_path+"/harm_primitives_headers.h"

<a id='introduction'></a>

# Step 1: Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

<a id='driver_conserv_to_prims'></a>

# Step 2: `driver_conserv_to_prims.C` \[Back to [top](#toc)\]
$$\label{driver_conserv_to_prims}$$

We start here by creating the `driver_conserv_to_prims.C` file and loading all files used by it. Note that of the files loaded, we have the following `IllinoisGRMHD` files:

1. `harm_primitives_headers.h`: we will discuss this file [in step 4 of this tutorial module](#harm_primitives_headers).
1. `inlined_functions.C`: this file is discussed in the [inlined_functions NRPy tutorial module](Tutorial-IllinoisGRMHD__inlined_functions.ipynb)
1. `apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C`: this file is discussed in the [apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs NRPy tutorial module](Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.ipynb)

In [2]:
%%writefile $outfile_path__driver_conserv_to_prims__C
/* We evolve forward in time a set of functions called the 
 *  "conservative variables", and any time the conserv's
 *  are updated, we must solve for the primitive variables 
 *  (rho, pressure, velocities) using a Newton-Raphson 
 *  technique, before reconstructing & evaluating the RHSs
 *  of the MHD equations again. 
 *
 * This file contains the driver routine for this Newton-
 *  Raphson solver. Truncation errors in conservative 
 *  variables can lead to no physical solutions in 
 *  primitive variables. We correct for these errors here 
 *  through a number of tricks described in the appendices 
 *  of http://arxiv.org/pdf/1112.0568.pdf.
 *
 * This is a wrapper for the 2d solver of Noble et al. See 
 *  harm_utoprim_2d.c for references and copyright notice 
 *  for that solver. This wrapper was primarily written by
 *  Zachariah Etienne & Yuk Tung Liu, in 2011-2013.
 * 
 * For optimal compatibility, this wrapper is licensed under 
 *  the GPL v2 or any later version.
 *
 * Note that this code assumes a simple gamma law for the 
 *  moment, though it would be easy to extend to a piecewise
 *  polytrope. */


#include "cctk.h"
#include <iostream>
#include <iomanip>
#include <fstream>
#include <sys/time.h>
#include <cmath>
#include <ctime>
#include <cstdlib>
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"
#include "Symmetry.h"

#include "IllinoisGRMHD_headers.h"
#include "harm_primitives_headers.h"
#include "inlined_functions.C"
#include "apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C"

extern "C" void IllinoisGRMHD_conserv_to_prims(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  // We use proper C++ here, for file I/O later.
  using namespace std;

  // Leo says: This is definitely one of the places we need to look at later.
  //           Will leave this comment to emphasize the "FIXME" below.
  // FIXME: only for single gamma-law EOS.
  eos_struct eos;
  eos.neos=neos;
  eos.K_poly=K_poly;
  eos.rho_tab[0]=rho_tab[0];
  eos.P_tab[0]=P_tab[0];
  eos.gamma_th=gamma_th;
  eos.eps_tab[0]=eps_tab[0];
  eos.k_tab[0]=k_tab[0];   eos.k_tab[1]=k_tab[1];
  eos.gamma_tab[0]=gamma_tab[0]; eos.gamma_tab[1]=gamma_tab[1];

Overwriting ../src/driver_conserv_to_prims.C


<a id='adm_to_bssn__enforcing_detgammabar_equal_one'></a>

## Step 2.a: Converting ADM quantities to BSSN quantities and enforcing $\bar\gamma=1$ \[Back to [top](#toc)\]
$$\label{adm_to_bssn__enforcing_detgammabar_equal_one}$$

Here we compute the conformal metric, $\bar\gamma_{ij}$, and its inverse, $\bar\gamma^{ij}$, from the physical metric $\gamma_{ij}$. We also compute $\phi$, the conformal factor, and $\psi\equiv e^{\phi}$. Finally, we enforce the constraint $\bar\gamma = \det\left(\bar\gamma_{ij}\right) = 1$. The entire procedure is explained in detail in the [convert ADM to BSSN and enforce determinant constraint NRPy tutorial module](Tutorial-IllinoisGRMHD__convert_ADM_to_BSSN__enforce_detgtij_eq_1__and_compute_gtupij.ipynb).

In [3]:
%%writefile -a $outfile_path__driver_conserv_to_prims__C


  // These BSSN-based variables are not evolved, and so are not defined anywhere that the grid has moved.
  // Here we convert ADM variables (from ADMBase) to the BSSN-based variables expected by this routine.
  IllinoisGRMHD_convert_ADM_to_BSSN__enforce_detgtij_eq_1__and_compute_gtupij(cctkGH,cctk_lsh,  gxx,gxy,gxz,gyy,gyz,gzz,alp,
                                                                gtxx,gtxy,gtxz,gtyy,gtyz,gtzz,
                                                                gtupxx,gtupxy,gtupxz,gtupyy,gtupyz,gtupzz,
                                                                phi_bssn,psi_bssn,lapm1);

Appending to ../src/driver_conserv_to_prims.C


<a id='equatorial_symmetry'></a>

## Step 2.b: Applying equatorial symmetry \[Back to [top](#toc)\]
$$\label{equatorial_symmetry}$$

We then use the [CardGrid3D ETK thorn](https://einsteintoolkit.org/thornguide/CactusBase/CartGrid3D/documentation.html) to apply equatorial symmetry to our problem.

In [4]:
%%writefile -a $outfile_path__driver_conserv_to_prims__C


  if(CCTK_EQUALS(Symmetry,"equatorial")) {
    // SET SYMMETRY GHOSTZONES ON ALL CONSERVATIVE VARIABLES!
    int ierr=0;
    ierr+=CartSymGN(cctkGH,"IllinoisGRMHD::grmhd_conservatives");
    // FIXME: UGLY. Filling metric ghostzones is needed for, e.g., Cowling runs.
    ierr+=CartSymGN(cctkGH,"lapse::lapse_vars");
    ierr+=CartSymGN(cctkGH,"bssn::BSSN_vars");
    ierr+=CartSymGN(cctkGH,"bssn::BSSN_AH");
    ierr+=CartSymGN(cctkGH,"shift::shift_vars");
    if(ierr!=0) CCTK_VError(VERR_DEF_PARAMS,"IllinoisGRMHD ERROR (grep for it, foo!)  :(");
  }

Appending to ../src/driver_conserv_to_prims.C


<a id='variable_setup'></a>

## Step 2.c: Setting up the variables needed by `HARM` \[Back to [top](#toc)\]
$$\label{variable_setup}$$

We will now set up all the necessary variables to start the conservative to primitive algorithm. We begin by declaring useful debugging variables.

In [5]:
%%writefile -a $outfile_path__driver_conserv_to_prims__C


  //Start the timer, so we can benchmark the primitives solver during evolution.
  //  Slower solver -> harder to find roots -> things may be going crazy!
  //FIXME: Replace this timing benchmark with something more meaningful, like the avg # of Newton-Raphson iterations per gridpoint!
  /*
    struct timeval start, end;
    long mtime, seconds, useconds;
    gettimeofday(&start, NULL);
  */

  int gamma_equals2 = 1;
  if (fabs(gamma_th-2.0) > 1.e-10) gamma_equals2 = 0;

  int failures=0,font_fixes=0,vel_limited_ptcount=0;
  int pointcount=0;
  int failures_inhoriz=0;
  int pointcount_inhoriz=0;

  int pressure_cap_hit=0;

  CCTK_REAL error_int_numer=0,error_int_denom=0;

  int imin=0,jmin=0,kmin=0;
  int imax=cctk_lsh[0],jmax=cctk_lsh[1],kmax=cctk_lsh[2];

  int rho_star_fix_applied=0;
  long n_iter=0;

Appending to ../src/driver_conserv_to_prims.C


<a id='variable_setup__bssn'></a>

### Step 2.c.i: BSSN quantities \[Back to [top](#toc)\]
$$\label{variable_setup__bssn}$$

We start by loading the BSSN variables $\left\{\phi, \bar\gamma_{ij}, \alpha-1, \beta^{i}, \bar\gamma^{ij}\right\}$ into a new array called $\rm METRIC$.

In [6]:
%%writefile -a $outfile_path__driver_conserv_to_prims__C

#pragma omp parallel for reduction(+:failures,vel_limited_ptcount,font_fixes,pointcount,failures_inhoriz,pointcount_inhoriz,error_int_numer,error_int_denom,pressure_cap_hit,rho_star_fix_applied,n_iter) schedule(static)
  for(int k=kmin;k<kmax;k++)
    for(int j=jmin;j<jmax;j++)
      for(int i=imin;i<imax;i++) {
        int index = CCTK_GFINDEX3D(cctkGH,i,j,k);

        int ww;
        CCTK_REAL METRIC[NUMVARS_FOR_METRIC],dummy=0;
        ww=0;
        // FIXME: NECESSARY?
        //psi_bssn[index] = exp(phi[index]);
        METRIC[ww] = phi_bssn[index];ww++;
        METRIC[ww] = dummy;          ww++; // Don't need to set psi.
        METRIC[ww] = gtxx[index];    ww++;
        METRIC[ww] = gtxy[index];    ww++;
        METRIC[ww] = gtxz[index];    ww++;
        METRIC[ww] = gtyy[index];    ww++;
        METRIC[ww] = gtyz[index];    ww++;
        METRIC[ww] = gtzz[index];    ww++;
        METRIC[ww] = lapm1[index];   ww++;
        METRIC[ww] = betax[index];   ww++;
        METRIC[ww] = betay[index];   ww++;
        METRIC[ww] = betaz[index];   ww++;
        METRIC[ww] = gtupxx[index];  ww++;
        METRIC[ww] = gtupyy[index];  ww++;
        METRIC[ww] = gtupzz[index];  ww++;
        METRIC[ww] = gtupxy[index];  ww++;
        METRIC[ww] = gtupxz[index];  ww++;
        METRIC[ww] = gtupyz[index];  ww++;

Appending to ../src/driver_conserv_to_prims.C


<a id='variable_setup__prims'></a>

### Step 2.c.ii: Primitives \[Back to [top](#toc)\]
$$\label{variable_setup__prims}$$

Then we load our current known values for the primitive variables $\left\{\rho_{b}, P, v^{i}, B^{i}\right\}$ into a new array called $\rm PRIMS$.

In [7]:
%%writefile -a $outfile_path__driver_conserv_to_prims__C


        CCTK_REAL PRIMS[MAXNUMVARS];
        ww=0;
        PRIMS[ww] = rho_b[index]; ww++;
        PRIMS[ww] = P[index];     ww++;
        PRIMS[ww] = vx[index];    ww++;
        PRIMS[ww] = vy[index];    ww++;
        PRIMS[ww] = vz[index];    ww++;
        PRIMS[ww] = Bx[index];    ww++;
        PRIMS[ww] = By[index];    ww++;
        PRIMS[ww] = Bz[index];    ww++;

Appending to ../src/driver_conserv_to_prims.C


<a id='variable_setup__conservs'></a>

### Step 2.c.iii: Conservatives \[Back to [top](#toc)\]
$$\label{variable_setup__conservs}$$

Then we load our current known values for the conservative variables $\left\{\rho_{\star}, \tilde{S}_{i}, \tilde{\tau}\right\}$ into a new array called $\rm CONSERVS$.

In [8]:
%%writefile -a $outfile_path__driver_conserv_to_prims__C


        CCTK_REAL CONSERVS[NUM_CONSERVS] = {rho_star[index], mhd_st_x[index],mhd_st_y[index],mhd_st_z[index],tau[index]};

Appending to ../src/driver_conserv_to_prims.C


<a id='variable_setup__lapse_and_psi'></a>

### Step 2.c.iv: Lapse function and conformal factor \[Back to [top](#toc)\]
$$\label{variable_setup__lapse_and_psi}$$

Then we load the lapse function, $\alpha$, and $\psi$ related variables into the $\rm METRIC\_LAP\_PSI4$ array. Notice that this is done using the ${\rm SET\_LAPSE\_PSI4}()$ "function", which is defined in the `IllinoisGRMHD_headers.h` file from `IllinoisGRMHD`. The "function" itself is quite simple:

```c
#define SET_LAPSE_PSI4(array_name,METRIC)   {                   \
      array_name[LAPSE] = METRIC[LAPM1]+1.0;                    \
      array_name[PSI2]  = exp(2.0*METRIC[PHI]);                 \
      array_name[PSI4]  = SQR(array_name[PSI2]);                \
      array_name[PSI6]  = array_name[PSI4]*array_name[PSI2];    \
      array_name[PSIM4]  = 1.0/array_name[PSI4];                \
      array_name[LAPSEINV]  = 1.0/array_name[LAPSE];            \
  }
```

defining the quantities $\left\{\alpha, \psi^{2}, \psi^{4}, \psi^{6}, \psi^{-4},\alpha^{-1}\right\}$, where $\psi=e^{\phi}$.

In [9]:
%%writefile -a $outfile_path__driver_conserv_to_prims__C


        CCTK_REAL METRIC_LAP_PSI4[NUMVARS_METRIC_AUX];
        SET_LAPSE_PSI4(METRIC_LAP_PSI4,METRIC);

Appending to ../src/driver_conserv_to_prims.C


<a id='variable_setup__phys_metric'></a>

### Step 2.c.v: Physical spatial metric \[Back to [top](#toc)\]
$$\label{variable_setup__phys_metric}$$

Then we set up the physical spatial metric and its inverse through the relations

$$
\boxed{
\begin{align}
\gamma_{ij} &= \psi^{4} \bar\gamma_{ij}\\
\gamma^{ij} &= \psi^{-4}\bar\gamma^{ij}
\end{align}
}\ .
$$

In [10]:
%%writefile -a $outfile_path__driver_conserv_to_prims__C

    
        CCTK_REAL METRIC_PHYS[NUMVARS_FOR_METRIC];
        METRIC_PHYS[GXX]   = METRIC[GXX]*METRIC_LAP_PSI4[PSI4];
        METRIC_PHYS[GXY]   = METRIC[GXY]*METRIC_LAP_PSI4[PSI4];
        METRIC_PHYS[GXZ]   = METRIC[GXZ]*METRIC_LAP_PSI4[PSI4];
        METRIC_PHYS[GYY]   = METRIC[GYY]*METRIC_LAP_PSI4[PSI4];
        METRIC_PHYS[GYZ]   = METRIC[GYZ]*METRIC_LAP_PSI4[PSI4];
        METRIC_PHYS[GZZ]   = METRIC[GZZ]*METRIC_LAP_PSI4[PSI4];
        METRIC_PHYS[GUPXX] = METRIC[GUPXX]*METRIC_LAP_PSI4[PSIM4];
        METRIC_PHYS[GUPXY] = METRIC[GUPXY]*METRIC_LAP_PSI4[PSIM4];
        METRIC_PHYS[GUPXZ] = METRIC[GUPXZ]*METRIC_LAP_PSI4[PSIM4];
        METRIC_PHYS[GUPYY] = METRIC[GUPYY]*METRIC_LAP_PSI4[PSIM4];
        METRIC_PHYS[GUPYZ] = METRIC[GUPYZ]*METRIC_LAP_PSI4[PSIM4];
        METRIC_PHYS[GUPZZ] = METRIC[GUPZZ]*METRIC_LAP_PSI4[PSIM4];

Appending to ../src/driver_conserv_to_prims.C


<a id='variable_setup__betadown_and_beta2'></a>

### Step 2.c.vi: $\beta_{i}$ and $\beta^{2} \equiv \beta_{i}\beta^{i}$ \[Back to [top](#toc)\]
$$\label{variable_setup__betadown_and_beta2}$$

We then evaluate

$$
\beta_{i} = \gamma_{ij}\beta^{j} \implies
\boxed{
\left\{
\begin{align}
\beta_{x} &= \gamma_{xx}\beta^{x} + \gamma_{xy}\beta^{y} + \gamma_{xz}\beta^{z}\\
\beta_{y} &= \gamma_{yx}\beta^{x} + \gamma_{yy}\beta^{y} + \gamma_{yz}\beta^{z}\\
\beta_{z} &= \gamma_{zx}\beta^{x} + \gamma_{zy}\beta^{y} + \gamma_{zz}\beta^{z}
\end{align}
\right.
}\ ,
$$

and

$$
\boxed{\beta^{2} \equiv \beta_{i}\beta^{i} = \beta_{x}\beta^{x} + \beta_{y}\beta^{y} + \beta_{z}\beta^{z}}\ .
$$

In [11]:
%%writefile -a $outfile_path__driver_conserv_to_prims__C


        CCTK_REAL TUPMUNU[10],TDNMUNU[10];

        CCTK_REAL shift_xL = METRIC_PHYS[GXX]*METRIC[SHIFTX] + METRIC_PHYS[GXY]*METRIC[SHIFTY] + METRIC_PHYS[GXZ]*METRIC[SHIFTZ];
        CCTK_REAL shift_yL = METRIC_PHYS[GXY]*METRIC[SHIFTX] + METRIC_PHYS[GYY]*METRIC[SHIFTY] + METRIC_PHYS[GYZ]*METRIC[SHIFTZ];
        CCTK_REAL shift_zL = METRIC_PHYS[GXZ]*METRIC[SHIFTX] + METRIC_PHYS[GYZ]*METRIC[SHIFTY] + METRIC_PHYS[GZZ]*METRIC[SHIFTZ];
        CCTK_REAL beta2L   = shift_xL*METRIC[SHIFTX] + shift_yL*METRIC[SHIFTY] + shift_zL*METRIC[SHIFTZ];

Appending to ../src/driver_conserv_to_prims.C


<a id='variable_setup__adm_4metric'></a>

### Step 2.c.vii: The ADM 4-metric, $g_{\mu\nu}$, and its inverse, $g^{\mu\nu}$ \[Back to [top](#toc)\]
$$\label{variable_setup__adm_4metric}$$

We then setup the ADM 4-metric and its inverse. We refer the reader to eqs. (2.119) and (2.122) from [Baumgarte & Shapiro's Numerical Relativity (2010)](https://books.google.com/books/about/Numerical_Relativity.html?id=dxU1OEinvRUC), which are repeated here for the sake of the reader (in reverse order)

$$
\boxed{g_{\mu\nu} =
\begin{pmatrix}
-\alpha^{2} + \beta_{\ell}\beta^{\ell} & \beta_{i}\\
\beta_{j} & \gamma_{ij}
\end{pmatrix}}\ .
$$

and

$$
\boxed{
g^{\mu\nu} =
\begin{pmatrix}
-\alpha^{-2} & \alpha^{-2}\beta^{i}\\
\alpha^{-2}\beta^{j} & \gamma^{ij} - \alpha^{-2}\beta^{i}\beta^{j}
\end{pmatrix}
}\ .
$$

In [12]:
%%writefile -a $outfile_path__driver_conserv_to_prims__C


        // Compute 4-metric, both g_{\mu \nu} and g^{\mu \nu}.
        // This is for computing T_{\mu \nu} and T^{\mu \nu}. Also the HARM con2prim lowlevel function requires them.
        CCTK_REAL g4dn[4][4],g4up[4][4];
        g4dn[0][0] = -SQR(METRIC_LAP_PSI4[LAPSE]) + beta2L;
        g4dn[0][1] = g4dn[1][0] = shift_xL;
        g4dn[0][2] = g4dn[2][0] = shift_yL;
        g4dn[0][3] = g4dn[3][0] = shift_zL;
        g4dn[1][1]              = METRIC_PHYS[GXX];
        g4dn[1][2] = g4dn[2][1] = METRIC_PHYS[GXY];
        g4dn[1][3] = g4dn[3][1] = METRIC_PHYS[GXZ];
        g4dn[2][2]              = METRIC_PHYS[GYY];
        g4dn[2][3] = g4dn[3][2] = METRIC_PHYS[GYZ];
        g4dn[3][3]              = METRIC_PHYS[GZZ];
        
        CCTK_REAL alpha_inv_squared=SQR(METRIC_LAP_PSI4[LAPSEINV]);
        g4up[0][0] = -1.0*alpha_inv_squared;
        g4up[0][1] = g4up[1][0] = METRIC[SHIFTX]*alpha_inv_squared;
        g4up[0][2] = g4up[2][0] = METRIC[SHIFTY]*alpha_inv_squared;
        g4up[0][3] = g4up[3][0] = METRIC[SHIFTZ]*alpha_inv_squared;
        g4up[1][1]              = METRIC_PHYS[GUPXX] - METRIC[SHIFTX]*METRIC[SHIFTX]*alpha_inv_squared;
        g4up[1][2] = g4up[2][1] = METRIC_PHYS[GUPXY] - METRIC[SHIFTX]*METRIC[SHIFTY]*alpha_inv_squared;
        g4up[1][3] = g4up[3][1] = METRIC_PHYS[GUPXZ] - METRIC[SHIFTX]*METRIC[SHIFTZ]*alpha_inv_squared;
        g4up[2][2]              = METRIC_PHYS[GUPYY] - METRIC[SHIFTY]*METRIC[SHIFTY]*alpha_inv_squared;
        g4up[2][3] = g4up[3][2] = METRIC_PHYS[GUPYZ] - METRIC[SHIFTY]*METRIC[SHIFTZ]*alpha_inv_squared;
        g4up[3][3]              = METRIC_PHYS[GUPZZ] - METRIC[SHIFTZ]*METRIC[SHIFTZ]*alpha_inv_squared;

Appending to ../src/driver_conserv_to_prims.C


<a id='variable_setup__temp_conservs'></a>

### Step 2.c.viii: Temporary storage for current values of the conservative variables \[Back to [top](#toc)\]
$$\label{variable_setup__temp_conservs}$$

Instead of declaring new variables to store the currently known values of the conservative variables $\left\{\rho_{\star}, \tilde{S}_{i}, \tilde{\tau}\right\}$, we will simply use the flux variables $\left\{\rho_{\star}^{\rm flux}, \tilde{S}_{i}^{\rm flux}, \tilde{\tau}^{\rm flux}\right\}$ which are used by `IllinoisGRMHD` as temporary storage. This is done for debugging purposes.

We also store the original values in the variables $\left\{\rho_{\star}^{\rm orig}, \tilde{S}_{i}^{\rm orig}, \tilde{\tau}^{\rm orig}\right\}$.

In [13]:
%%writefile -a $outfile_path__driver_conserv_to_prims__C


        //FIXME: might slow down the code.
        if(isnan(CONSERVS[RHOSTAR]*CONSERVS[STILDEX]*CONSERVS[STILDEY]*CONSERVS[STILDEZ]*CONSERVS[TAUENERGY]*PRIMS[BX_CENTER]*PRIMS[BY_CENTER]*PRIMS[BZ_CENTER])) {
          CCTK_VInfo(CCTK_THORNSTRING,"NAN FOUND: i,j,k = %d %d %d, x,y,z = %e %e %e , index=%d st_i = %e %e %e, rhostar = %e, tau = %e, Bi = %e %e %e, gij = %e %e %e %e %e %e, Psi6 = %e",
                     i,j,k,x[index],y[index],z[index],index,
                     CONSERVS[STILDEX],CONSERVS[STILDEY],CONSERVS[STILDEZ],CONSERVS[RHOSTAR],CONSERVS[TAUENERGY],
                     PRIMS[BX_CENTER],PRIMS[BY_CENTER],PRIMS[BZ_CENTER],METRIC_PHYS[GXX],METRIC_PHYS[GXY],METRIC_PHYS[GXZ],METRIC_PHYS[GYY],METRIC_PHYS[GYZ],METRIC_PHYS[GZZ],METRIC_LAP_PSI4[PSI6]);
        }
        
        // Here we use _flux variables as temp storage for original values of conservative variables.. This is used for debugging purposes only.
        rho_star_flux[index] = CONSERVS[RHOSTAR];
        st_x_flux[index]     = CONSERVS[STILDEX];
        st_y_flux[index]     = CONSERVS[STILDEY];
        st_z_flux[index]     = CONSERVS[STILDEZ];
        tau_flux[index]      = CONSERVS[TAUENERGY];

        CCTK_REAL rho_star_orig = CONSERVS[RHOSTAR];
        CCTK_REAL mhd_st_x_orig = CONSERVS[STILDEX];
        CCTK_REAL mhd_st_y_orig = CONSERVS[STILDEY];
        CCTK_REAL mhd_st_z_orig = CONSERVS[STILDEZ];
        CCTK_REAL tau_orig      = CONSERVS[TAUENERGY];

Appending to ../src/driver_conserv_to_prims.C


<a id='conserv_to_prim__driver'></a>

## Step 2.d: Determining the primitives variables from the conservatives variables \[Back to [top](#toc)\]
$$\label{conserv_to_prim__driver}$$

In this part of the code we determine the primitives variables from the conservatives variables. Please note that this is only the driver function. The algorithm is discussed on [Step 3](#harm_primitives_lowlevel) below.

We start by calling the `apply_tau_floor()` function to ensure that the value of $\tilde\tau$ is not out of physical range. This function is discussed in the [apply $\tilde\tau$ floor and enforce limits on primitives NRPy+ tutorial module](Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.ipynb).

Notice that this is only performed when $\rho_{\star}>0$. If this is not the case, we fix the issue by setting $\rho_{b} = \rho_{b}^{\rm atm}$ and the conservative to primitive algorithm is skipped altogether.

When $\rho_{\star}>0$, we call upon the primitive to conservatives algorithm through the `harm_primitives_gammalaw_lowlevel()` function, described [below](#fixme).

In [14]:
%%writefile -a $outfile_path__driver_conserv_to_prims__C


        int check=0;
        struct output_stats stats;
        stats.n_iter=0;
        stats.vel_limited=0;
        stats.failure_checker=0;

        if(CONSERVS[RHOSTAR]>0.0) {
          // Apply the tau floor
          apply_tau_floor(index,tau_atm,rho_b_atm,Psi6threshold,PRIMS,METRIC,METRIC_PHYS,METRIC_LAP_PSI4,stats,eos,  CONSERVS);

          stats.font_fixed=0;
          for(int ii=0;ii<3;ii++) {
            check = harm_primitives_gammalaw_lowlevel(index,i,j,k,x,y,z,METRIC,METRIC_PHYS,METRIC_LAP_PSI4,
                                                      CONSERVS,PRIMS,  g4dn,g4up,   stats,eos);
            if(check==0) ii=4;
            else stats.failure_checker+=100000;
          }
        } else {
          stats.failure_checker+=1;
          // Set to atmosphere if rho_star<0.
          //FIXME: FOR GAMMA=2 ONLY:
          PRIMS[RHOB]      =rho_b_atm;
          if(gamma_equals2) {
            PRIMS[PRESSURE]=K_poly*SQR(rho_b_atm);
          } else {
            PRIMS[PRESSURE]=K_poly*pow(rho_b_atm,gamma_th);
          }
          PRIMS[VX]        =-METRIC[SHIFTX];
          PRIMS[VY]        =-METRIC[SHIFTY];
          PRIMS[VZ]        =-METRIC[SHIFTZ];

          rho_star_fix_applied++;
        }

Appending to ../src/driver_conserv_to_prims.C


<a id='enforce_limits_on_primitives_and_recompute_conservs'></a>

## Step 2.e: Enforcing physical limits on primitives and recomputing the conservatives variables \[Back to [top](#toc)\]
$$\label{enforce_limits_on_primitives_and_recompute_conservs}$$

We now enforce that the values of the primitives variables are physically meaningful. This is done by calling the `IllinoisGRMHD_enforce_limits_on_primitives_and_recompute_conservs()` function, which is described in the [enforce physical limits on primitives and recomputive conservatives NRPy+ tutorial module](Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.ipynb).

In [15]:
%%writefile -a $outfile_path__driver_conserv_to_prims__C


        // Enforce limits on primitive variables and recompute conservatives.
        static const int already_computed_physical_metric_and_inverse=1;
        IllinoisGRMHD_enforce_limits_on_primitives_and_recompute_conservs(already_computed_physical_metric_and_inverse,PRIMS,stats,eos,METRIC,g4dn,g4up, TUPMUNU,TDNMUNU,CONSERVS);

Appending to ../src/driver_conserv_to_prims.C


<a id='updating_conservs_and_prims_gfs'></a>

## Step 2.f: Updating conservative and primitive gridfunctions \[Back to [top](#toc)\]
$$\label{updating_conservs_and_prims_gfs}$$

Then we update the corresponding conservative and primitive gridfunctions with the updated values we just computed.

In [16]:
%%writefile -a $outfile_path__driver_conserv_to_prims__C


        rho_star[index] = CONSERVS[RHOSTAR];
        mhd_st_x[index] = CONSERVS[STILDEX];
        mhd_st_y[index] = CONSERVS[STILDEY];
        mhd_st_z[index] = CONSERVS[STILDEZ];
        tau[index] = CONSERVS[TAUENERGY];

        // Set primitives, and/or provide a better guess.
        rho_b[index] = PRIMS[RHOB];
        P[index]     = PRIMS[PRESSURE];
        vx[index]    = PRIMS[VX];
        vy[index]    = PRIMS[VY];
        vz[index]    = PRIMS[VZ];

Appending to ../src/driver_conserv_to_prims.C


<a id='diagnostics_and_debugging_tools'></a>

## Step 2.g: Diagnostics and debugging tools \[Back to [top](#toc)\]
$$\label{diagnostics_and_debugging_tools}$$

Now we simply append to the file useful diagnostics and debugging tools for our code.

In [17]:
%%writefile -a $outfile_path__driver_conserv_to_prims__C


        if(update_Tmunu) {
          ww=0;
          eTtt[index] = TDNMUNU[ww]; ww++;
          eTtx[index] = TDNMUNU[ww]; ww++;
          eTty[index] = TDNMUNU[ww]; ww++;
          eTtz[index] = TDNMUNU[ww]; ww++;
          eTxx[index] = TDNMUNU[ww]; ww++;
          eTxy[index] = TDNMUNU[ww]; ww++;
          eTxz[index] = TDNMUNU[ww]; ww++;
          eTyy[index] = TDNMUNU[ww]; ww++;
          eTyz[index] = TDNMUNU[ww]; ww++;
          eTzz[index] = TDNMUNU[ww];
        }

        //Finally, we set h, the enthalpy:
        //CCTK_REAL eps = P[index]/rho_b[index]/(GAMMA-1.0);
        //h[index] = 1.0 + P[index]/rho_b[index] + eps;

        /***************************************************************************************************************************/
        // DIAGNOSTICS:
        //Pressure cap hit?
        /* FIXME
        CCTK_REAL P_cold = rho_b[index]*rho_b[index];
        if(P[index]/P_cold > 0.99*1e3 && rho_b[index]>100.0*rho_b_atm) {
          if(exp(phi[index]*6.0) <= Psi6threshold) pressure_cap_hit++;
        }
        */

        //Now we compute the difference between original & new conservatives, for diagnostic purposes:
        error_int_numer += fabs(tau[index] - tau_orig) + fabs(rho_star[index] - rho_star_orig) + 
          fabs(mhd_st_x[index] - mhd_st_x_orig) + fabs(mhd_st_y[index] - mhd_st_y_orig) + fabs(mhd_st_z[index] - mhd_st_z_orig);
        error_int_denom += tau_orig + rho_star_orig + fabs(mhd_st_x_orig) + fabs(mhd_st_y_orig) + fabs(mhd_st_z_orig);

        if(stats.font_fixed==1) font_fixes++;
        vel_limited_ptcount+=stats.vel_limited;
        if(check!=0) {
          failures++;
          if(exp(METRIC[PHI]*6.0)>Psi6threshold) {
            failures_inhoriz++;
            pointcount_inhoriz++;
          }
        }
        pointcount++;
        /***************************************************************************************************************************/
        failure_checker[index] = stats.failure_checker;
        n_iter += stats.n_iter;
      }

  /*
    gettimeofday(&end, NULL);

    seconds  = end.tv_sec  - start.tv_sec;
    useconds = end.tv_usec - start.tv_usec;

    mtime = ((seconds) * 1000 + useconds/1000.0) + 0.999;  // We add 0.999 since mtime is a long int; this rounds up the result before setting the value.  Here, rounding down is incorrect.
    solutions per second: cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2] / ((CCTK_REAL)mtime/1000.0),
  */
  if(CCTK_Equals(verbose, "essential") || CCTK_Equals(verbose, "essential+iteration output")) {
    CCTK_VInfo(CCTK_THORNSTRING,"C2P: Lev: %d NumPts= %d | Fixes: Font= %d VL= %d rho*= %d | Failures: %d InHoriz= %d / %d | Error: %.3e, ErrDenom: %.3e | %.2f iters/gridpt",
               (int)GetRefinementLevel(cctkGH),
               pointcount,font_fixes,vel_limited_ptcount,rho_star_fix_applied,
               failures,
               failures_inhoriz,pointcount_inhoriz,
               error_int_numer/error_int_denom,error_int_denom,
               (double)n_iter/( (double)(cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2]) ));
  }
  
  if(pressure_cap_hit!=0) {
    //CCTK_VInfo(CCTK_THORNSTRING,"PRESSURE CAP HIT %d TIMES!  Outputting debug file!",pressure_cap_hit);
  }

  // Very useful con2prim debugger. If the primitives (con2prim) solver fails, this will output all data needed to 
  //     debug where and why the solver failed. Strongly suggested for experimenting with new fixes.
  if(conserv_to_prims_debug==1) {

    ofstream myfile;
    char filename[100];
    srand(time(NULL));
    sprintf(filename,"primitives_debug-%e.dat",rho_star[CCTK_GFINDEX3D(cctkGH,3,15,6)]);
    //Alternative, for debugging purposes as well: sprintf(filename,"primitives_debug-%d.dat",rand());
    myfile.open (filename, ios::out | ios::binary);
    //myfile.open ("data.bin", ios::out | ios::binary);
    myfile.write((char*)cctk_lsh, 3*sizeof(int));

    myfile.write((char*)&rho_b_atm, 1*sizeof(CCTK_REAL));
    myfile.write((char*)&tau_atm, 1*sizeof(CCTK_REAL));

    myfile.write((char*)&Psi6threshold, 1*sizeof(CCTK_REAL));

    CCTK_REAL gamma_th=2.0;
    myfile.write((char*)&gamma_th, 1*sizeof(CCTK_REAL));
    myfile.write((char*)&neos,     1*sizeof(int));

    myfile.write((char*)gamma_tab, (neos+1)*sizeof(CCTK_REAL));
    myfile.write((char*)k_tab,     (neos+1)*sizeof(CCTK_REAL));

    myfile.write((char*)eps_tab,   neos*sizeof(CCTK_REAL));
    myfile.write((char*)rho_tab,   neos*sizeof(CCTK_REAL));
    myfile.write((char*)P_tab,     neos*sizeof(CCTK_REAL));

    int fullsize=cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2];
    myfile.write((char*)x,   (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)y,   (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)z,   (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)phi_bssn, (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)gtxx, (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)gtxy, (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)gtxz, (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)gtyy, (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)gtyz, (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)gtzz, (fullsize)*sizeof(CCTK_REAL));

    myfile.write((char*)gtupxx, (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)gtupxy, (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)gtupxz, (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)gtupyy, (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)gtupyz, (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)gtupzz, (fullsize)*sizeof(CCTK_REAL));

    myfile.write((char*)betax, (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)betay, (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)betaz, (fullsize)*sizeof(CCTK_REAL));

    myfile.write((char*)lapm1, (fullsize)*sizeof(CCTK_REAL));
 
    // HERE WE USE _flux variables as temp storage for original values of conservative variables.. This is used for debugging purposes only.
    myfile.write((char*)tau_flux,      (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)st_x_flux, (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)st_y_flux, (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)st_z_flux, (fullsize)*sizeof(CCTK_REAL));

    myfile.write((char*)rho_star_flux, (fullsize)*sizeof(CCTK_REAL));

    myfile.write((char*)Bx,   (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)By,   (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)Bz,   (fullsize)*sizeof(CCTK_REAL));

    myfile.write((char*)vx,   (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)vy,   (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)vz,   (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)P,    (fullsize)*sizeof(CCTK_REAL));
    myfile.write((char*)rho_b,(fullsize)*sizeof(CCTK_REAL));

    int checker=1063; myfile.write((char*)&checker,sizeof(int));

    myfile.close();
    CCTK_VInfo(CCTK_THORNSTRING,"Finished writing...");
  }
}

#include "harm_primitives_lowlevel.C"



Appending to ../src/driver_conserv_to_prims.C


<a id='harm_primitives_lowlevel'></a>

# Step 3: `harm_primitives_lowlevel.C` \[Back to [top](#toc)\]
$$\label{harm_primitives_lowlevel}$$

In [18]:
%%writefile $outfile_path__harm_primitives_lowlevel__C
inline int harm_primitives_gammalaw_lowlevel(const int index,const int i,const int j,const int k,CCTK_REAL *X,CCTK_REAL *Y,CCTK_REAL *Z,
                                             CCTK_REAL *METRIC,CCTK_REAL *METRIC_PHYS,CCTK_REAL *METRIC_LAP_PSI4,
                                             CCTK_REAL *CONSERVS,CCTK_REAL *PRIMS,
                                             CCTK_REAL g4dn[NDIM][NDIM],CCTK_REAL g4up[NDIM][NDIM],
                                             struct output_stats &stats, eos_struct &eos) {

  DECLARE_CCTK_PARAMETERS;

  CCTK_REAL kpoly = eos.k_tab[0];
  CCTK_REAL gamma = eos.gamma_tab[0];
  int gamma_equals2 = 1;
  if (fabs(gamma-2.0) > 1.e-10) gamma_equals2 = 0;

  // declare some variables for HARM.
  CCTK_REAL U[NPR]; 
  CCTK_REAL prim[NPR];
  CCTK_REAL detg = METRIC_LAP_PSI4[LAPSE]*METRIC_LAP_PSI4[PSI6]; // == alpha sqrt{gamma} = alpha Psi^6

  // Check to see if the metric is positive-definite.
  // Note that this will slow down the code, and if the metric doesn't obey this, the run is probably too far gone to save,
  //   though if it happens deep in the horizon, it might resurrect the run.
  /*
    CCTK_REAL lam1,lam2,lam3;
    CCTK_REAL M11 = METRIC[GXX], M12=METRIC[GXY], M13=METRIC[GXZ], M22=METRIC[GYY], M23=METRIC[GYZ], M33=METRIC[GZZ];
    eigenvalues_3by3_real_sym_matrix(lam1, lam2, lam3,M11, M12, M13, M22, M23, M33);
    if (lam1 < 0.0 || lam2 < 0.0 || lam3 < 0.0) {
    // Metric is not positive-defitive, reset the physical metric to be conformally-flat.
    METRIC_PHYS[GXX] = METRIC_LAP_PSI4[PSI4];
    METRIC_PHYS[GXY] = 0.0;
    METRIC_PHYS[GXZ] = 0.0;
    METRIC_PHYS[GYY] = METRIC_LAP_PSI4[PSI4];
    METRIC_PHYS[GYZ] = 0.0;
    METRIC_PHYS[GZZ] = METRIC_LAP_PSI4[PSI4];
    METRIC_PHYS[GUPXX] = METRIC_LAP_PSI4[PSIM4];
    METRIC_PHYS[GUPXY] = 0.0;
    METRIC_PHYS[GUPXZ] = 0.0;
    METRIC_PHYS[GUPYY] = METRIC_LAP_PSI4[PSIM4];
    METRIC_PHYS[GUPYZ] = 0.0;
    METRIC_PHYS[GUPZZ] = METRIC_LAP_PSI4[PSIM4];
    }
  */

  // Note that ONE_OVER_SQRT_4PI gets us to the object
  // referred to as B^i in the Noble et al paper (and
  // apparently also in the comments to their code).
  // This is NOT the \mathcal{B}^i, which differs by 
  // a factor of the lapse.
  CCTK_REAL BxL_over_alpha_sqrt_fourpi = PRIMS[BX_CENTER]*METRIC_LAP_PSI4[LAPSEINV]*ONE_OVER_SQRT_4PI;
  CCTK_REAL ByL_over_alpha_sqrt_fourpi = PRIMS[BY_CENTER]*METRIC_LAP_PSI4[LAPSEINV]*ONE_OVER_SQRT_4PI;
  CCTK_REAL BzL_over_alpha_sqrt_fourpi = PRIMS[BZ_CENTER]*METRIC_LAP_PSI4[LAPSEINV]*ONE_OVER_SQRT_4PI;

  CCTK_REAL rho_b_oldL = PRIMS[RHOB];
  CCTK_REAL P_oldL     = PRIMS[PRESSURE];
  CCTK_REAL vxL        = PRIMS[VX];
  CCTK_REAL vyL        = PRIMS[VY];
  CCTK_REAL vzL        = PRIMS[VZ];

  /*
    -- Driver for new prim. var. solver.  The driver just translates
    between the two sets of definitions for U and P.  The user may 
    wish to alter the translation as they see fit.  


    //         /  rho u^t           \                             //
    //    U =  |  T^t_t   + rho u^t |  sqrt(-det(g_{\mu\nu}))     //
    //         |  T^t_i             |                             //
    //         \   B^i              /                             //
    //                                                            //
    //        /    rho        \                                   //
    //    P = |    uu         |                                   //
    //        | \tilde{u}^i   |                                   //
    //        \   B^i         /                                   //

    (above equations have been fixed by Yuk Tung & Zach)
  */
  
  // U[NPR]    = conserved variables (current values on input/output);
  // g4dn[NDIM][NDIM] = covariant form of the 4-metric ;
  // g4up[NDIM][NDIM] = contravariant form of the 4-metric ;
  // gdet             = sqrt( - determinant of the 4-metric) ;
  // prim[NPR] = primitive variables (guess on input, calculated values on 
  //                     output if there are no problems);

  // U[1]   =  
  // U[2-4] =  stildei + rhostar

  CCTK_REAL rho_star_orig = CONSERVS[RHOSTAR];
  CCTK_REAL mhd_st_x_orig = CONSERVS[STILDEX];
  CCTK_REAL mhd_st_y_orig = CONSERVS[STILDEY];
  CCTK_REAL mhd_st_z_orig = CONSERVS[STILDEZ];
  CCTK_REAL tau_orig      = CONSERVS[TAUENERGY];

  // Other ideas for setting the gamma speed limit
  //CCTK_REAL GAMMA_SPEED_LIMIT = 100.0;
  //if(METRIC_LAP_PSI4[PSI6]>Psi6threshold) GAMMA_SPEED_LIMIT=500.0;
  //if(METRIC_LAP_PSI4[PSI6]>Psi6threshold) GAMMA_SPEED_LIMIT=100.0;

  //FIXME: Only works if poisoning is turned on. Otherwise will access unknown memory. This trick alone speeds up the whole code (Cowling) by 2%.
  //int startguess=0;
  //if(std::isnan(PRIMS[VX])) startguess=1;
  int startguess=1;

  CCTK_REAL u0L=1.0;

  for(int which_guess=startguess;which_guess<3;which_guess++) {
    int check;

    if(which_guess==1) {
      //Use a different initial guess:
      rho_b_oldL = CONSERVS[RHOSTAR]/METRIC_LAP_PSI4[PSI6];
      if (gamma_equals2==1) {
        P_oldL = kpoly*rho_b_oldL*rho_b_oldL;
      } else {
        P_oldL = kpoly*pow(rho_b_oldL,gamma);
      }
      u0L = METRIC_LAP_PSI4[LAPSEINV];
      vxL = -METRIC[SHIFTX];
      vyL = -METRIC[SHIFTY];
      vzL = -METRIC[SHIFTZ];
    }

    if(which_guess==2) {
      //Use atmosphere as initial guess:
      rho_b_oldL = 100.0*rho_b_atm;
      // GAMMA=2 ONLY:
      if (gamma_equals2==1) {
        P_oldL = kpoly*rho_b_oldL*rho_b_oldL;
      } else {
        P_oldL = kpoly*pow(rho_b_oldL,gamma);
      }
      u0L = METRIC_LAP_PSI4[LAPSEINV];
      vxL = -METRIC[SHIFTX];
      vyL = -METRIC[SHIFTY];
      vzL = -METRIC[SHIFTZ];
    }

    // Fill the array of conserved variables according to the wishes of Utoprim_2d.
    U[RHO]    = CONSERVS[RHOSTAR];
    U[UU]     = -CONSERVS[TAUENERGY]*METRIC_LAP_PSI4[LAPSE] - (METRIC_LAP_PSI4[LAPSE]-1.0)*CONSERVS[RHOSTAR] + 
      METRIC[SHIFTX]*CONSERVS[STILDEX] + METRIC[SHIFTY]*CONSERVS[STILDEY]  + METRIC[SHIFTZ]*CONSERVS[STILDEZ] ; // note the minus sign on tau
    U[UTCON1] = CONSERVS[STILDEX];
    U[UTCON2] = CONSERVS[STILDEY];
    U[UTCON3] = CONSERVS[STILDEZ];
    U[BCON1]  = detg*BxL_over_alpha_sqrt_fourpi;
    U[BCON2]  = detg*ByL_over_alpha_sqrt_fourpi;
    U[BCON3]  = detg*BzL_over_alpha_sqrt_fourpi;

    CCTK_REAL uL = P_oldL/(gamma_th /* <- Should be local polytropic gamma factor */ - 1.0);
    CCTK_REAL utxL = u0L*(vxL + METRIC[SHIFTX]);
    CCTK_REAL utyL = u0L*(vyL + METRIC[SHIFTY]);
    CCTK_REAL utzL = u0L*(vzL + METRIC[SHIFTZ]);

    prim[RHO]    = rho_b_oldL;
    prim[UU]     = uL;
    prim[UTCON1] = utxL;
    prim[UTCON2] = utyL;
    prim[UTCON3] = utzL;
    prim[BCON1]  = BxL_over_alpha_sqrt_fourpi;
    prim[BCON2]  = ByL_over_alpha_sqrt_fourpi;
    prim[BCON3]  = BzL_over_alpha_sqrt_fourpi;

    /*************************************************************/
    // CALL HARM PRIMITIVES SOLVER:
    check = Utoprim_2d(U, g4dn, g4up, detg, prim,stats.n_iter);
    // Note that we have modified this solver, so that nearly 100% 
    // of the time it yields either a good root, or a root with 
    // negative epsilon (i.e., pressure).
    /*************************************************************/

    // Use the new Font fix subroutine 
    int font_fix_applied=0;
    if(check!=0) {
      font_fix_applied=1;
      CCTK_REAL u_xl=1e100, u_yl=1e100, u_zl=1e100; // Set to insane values to ensure they are overwritten.
      if (gamma_equals2==1) {
        check = font_fix_gamma_equals2(u_xl,u_yl,u_zl,CONSERVS,PRIMS,METRIC_PHYS,METRIC_LAP_PSI4,eos);
      } else {
        check = font_fix_general_gamma(u_xl,u_yl,u_zl,CONSERVS,PRIMS,METRIC_PHYS,METRIC_LAP_PSI4,eos);
      }
      //Translate to HARM primitive now:
      prim[UTCON1] = METRIC_PHYS[GUPXX]*u_xl + METRIC_PHYS[GUPXY]*u_yl + METRIC_PHYS[GUPXZ]*u_zl;
      prim[UTCON2] = METRIC_PHYS[GUPXY]*u_xl + METRIC_PHYS[GUPYY]*u_yl + METRIC_PHYS[GUPYZ]*u_zl;
      prim[UTCON3] = METRIC_PHYS[GUPXZ]*u_xl + METRIC_PHYS[GUPYZ]*u_yl + METRIC_PHYS[GUPZZ]*u_zl;
      if (check==1) {
        CCTK_VInfo(CCTK_THORNSTRING,"Font fix failed!");
        CCTK_VInfo(CCTK_THORNSTRING,"i,j,k = %d %d %d, stats.failure_checker = %d x,y,z = %e %e %e , index=%d st_i = %e %e %e, rhostar = %e, Bi = %e %e %e, gij = %e %e %e %e %e %e, Psi6 = %e",i,j,k,stats.failure_checker,X[index],Y[index],Z[index],index,mhd_st_x_orig,mhd_st_y_orig,mhd_st_z_orig,rho_star_orig,PRIMS[BX_CENTER],PRIMS[BY_CENTER],PRIMS[BZ_CENTER],METRIC_PHYS[GXX],METRIC_PHYS[GXY],METRIC_PHYS[GXZ],METRIC_PHYS[GYY],METRIC_PHYS[GYZ],METRIC_PHYS[GZZ],METRIC_LAP_PSI4[PSI6]);
      }
    }
    stats.failure_checker+=font_fix_applied*10000;
    stats.font_fixed=font_fix_applied;
    /*************************************************************/

    if(check==0) {
      //Now that we have found some solution, we first limit velocity:
      //FIXME: Probably want to use exactly the same velocity limiter function here as in mhdflux.C
      CCTK_REAL utx_new = prim[UTCON1];
      CCTK_REAL uty_new = prim[UTCON2];
      CCTK_REAL utz_new = prim[UTCON3];

      //Velocity limiter:
      CCTK_REAL gijuiuj = METRIC_PHYS[GXX]*SQR(utx_new ) +
        2.0*METRIC_PHYS[GXY]*utx_new*uty_new + 2.0*METRIC_PHYS[GXZ]*utx_new*utz_new +
        METRIC_PHYS[GYY]*SQR(uty_new) + 2.0*METRIC_PHYS[GYZ]*uty_new*utz_new +
        METRIC_PHYS[GZZ]*SQR(utz_new);
      CCTK_REAL au0m1 = gijuiuj/( 1.0+sqrt(1.0+gijuiuj) );
      u0L = (au0m1+1.0)*METRIC_LAP_PSI4[LAPSEINV];

      // *** Limit velocity
      stats.vel_limited=0;
      if (au0m1 > 0.9999999*(GAMMA_SPEED_LIMIT-1.0)) {
        CCTK_REAL fac = sqrt((SQR(GAMMA_SPEED_LIMIT)-1.0)/(SQR(1.0+au0m1) - 1.0));
        utx_new *= fac;
        uty_new *= fac;
        utz_new *= fac;
        gijuiuj = gijuiuj * SQR(fac);
        au0m1 = gijuiuj/( 1.0+sqrt(1.0+gijuiuj) );
        // Reset rho_b and u0
        u0L = (au0m1+1.0)*METRIC_LAP_PSI4[LAPSEINV];
        prim[RHO] =  rho_star_orig/(METRIC_LAP_PSI4[LAPSE]*u0L*METRIC_LAP_PSI4[PSI6]);
        stats.vel_limited=1;
        stats.failure_checker+=1000;
      } //Finished limiting velocity

      //The Font fix only sets the velocities.  Here we set the pressure & density HARM primitives.
      if(font_fix_applied==1) {
        prim[RHO] = rho_star_orig/(METRIC_LAP_PSI4[LAPSE]*u0L*METRIC_LAP_PSI4[PSI6]);
        //Next set P = P_cold:
        CCTK_REAL P_cold;
        if (gamma_equals2==1) {
          P_cold = kpoly*prim[RHO]*prim[RHO]; // <- FIXME: GAMMA=2 ONLY
        } else {
          P_cold = kpoly*pow(prim[RHO],gamma);
        }
        prim[UU] = P_cold/(gamma_th /* <- Should be local polytropic gamma factor */-1.0);
      } //Finished setting remaining primitives if there was a Font fix.

      PRIMS[RHOB]      = prim[RHO];
      PRIMS[PRESSURE] = (gamma_th /* <- Should be local polytropic gamma factor */-1.0)*prim[UU];
      //Already set u0L.
      PRIMS[VX]       = utx_new/u0L - METRIC[SHIFTX];
      PRIMS[VY]       = uty_new/u0L - METRIC[SHIFTY];
      PRIMS[VZ]       = utz_new/u0L - METRIC[SHIFTZ];

      return 0;
    } else {
      //If we didn't find a root, then try again with a different guess.
    }
  }
  CCTK_VInfo(CCTK_THORNSTRING,"Couldn't find root from: %e %e %e %e %e, rhob approx=%e, rho_b_atm=%e, Bx=%e, By=%e, Bz=%e, gij_phys=%e %e %e %e %e %e, alpha=%e",
	     tau_orig,rho_star_orig,mhd_st_x_orig,mhd_st_y_orig,mhd_st_z_orig,rho_star_orig/METRIC_LAP_PSI4[PSI6],rho_b_atm,PRIMS[BX_CENTER],PRIMS[BY_CENTER],PRIMS[BZ_CENTER],METRIC_PHYS[GXX],METRIC_PHYS[GXY],METRIC_PHYS[GXZ],METRIC_PHYS[GYY],METRIC_PHYS[GYZ],METRIC_PHYS[GZZ],METRIC_LAP_PSI4[LAPSE]);
  return 1;
}

#include "harm_u2p_util.c"
#include "harm_utoprim_2d.c"
#include "eigen.C"
#include "font_fix_gamma_law.C"



Overwriting ../src/harm_primitives_lowlevel.C


<a id='harm_primitives_headers'></a>

# Step 4: `harm_primitives_headers.h` \[Back to [top](#toc)\]
$$\label{harm_primitives_headers}$$

In [19]:
%%writefile $outfile_path__harm_primitives_headers__h
/***********************************************************************************
    Copyright 2006 Charles F. Gammie, Jonathan C. McKinney, Scott C. Noble, 
                   Gabor Toth, and Luca Del Zanna

                        HARM  version 1.0   (released May 1, 2006)

    This file is part of HARM.  HARM is a program that solves hyperbolic 
    partial differential equations in conservative form using high-resolution
    shock-capturing techniques.  This version of HARM has been configured to 
    solve the relativistic magnetohydrodynamic equations of motion on a 
    stationary black hole spacetime in Kerr-Schild coordinates to evolve
    an accretion disk model. 

    You are morally obligated to cite the following two papers in his/her 
    scientific literature that results from use of any part of HARM:

    [1] Gammie, C. F., McKinney, J. C., \& Toth, G.\ 2003, 
        Astrophysical Journal, 589, 444.

    [2] Noble, S. C., Gammie, C. F., McKinney, J. C., \& Del Zanna, L. \ 2006, 
        Astrophysical Journal, 641, 626.

   
    Further, we strongly encourage you to obtain the latest version of 
    HARM directly from our distribution website:
    http://rainman.astro.uiuc.edu/codelib/


    HARM is free software; you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation; either version 2 of the License, or
    (at your option) any later version.

    HARM is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with HARM; if not, write to the Free Software
    Foundation, Inc., 51 Franklin St, Fifth Floor, Boston, MA  02110-1301  USA

***********************************************************************************/
#ifndef HARM_PRIMITIVES_HEADERS_H_
#define HARM_PRIMITIVES_HEADERS_H_

static const int NPR =8;
static const int NDIM=4;

/* Adiabatic index used for the state equation */
//#define GAMMA    (2.0)

static const CCTK_REAL G_ISOTHERMAL = 1.0;

/* use K(s)=K(r)=const. (G_ATM = GAMMA) of time or  T = T(r) = const. of time (G_ATM = 1.) */
/*
  #define USE_ISENTROPIC 1

  #if( USE_ISENTROPIC ) 
  #define G_ATM GAMMA
  #else
  #define G_ATM G_ISOTHERMAL
  #endif
*/

static const int MAX_NEWT_ITER=30;     /* Max. # of Newton-Raphson iterations for find_root_2D(); */
//#define MAX_NEWT_ITER 300     /* Max. # of Newton-Raphson iterations for find_root_2D(); */
static const CCTK_REAL NEWT_TOL    =1.0e-10;    /* Min. of tolerance allowed for Newton-Raphson iterations */
static const CCTK_REAL MIN_NEWT_TOL=1.0e-10;    /* Max. of tolerance allowed for Newton-Raphson iterations */
static const int EXTRA_NEWT_ITER=0; /* ZACH SAYS: Original value = 2. But I don't think this parameter > 0 is warranted. Just slows the code for no reason, since our tolerances are fine. */

static const CCTK_REAL NEWT_TOL2    =1.0e-15;      /* TOL of new 1D^*_{v^2} gnr2 method */
static const CCTK_REAL MIN_NEWT_TOL2=1.0e-10;  /* TOL of new 1D^*_{v^2} gnr2 method */

static const CCTK_REAL W_TOO_BIG    =1.e20;    /* \gamma^2 (\rho_0 + u + p) is assumed
                                                  to always be smaller than this.  This
                                                  is used to detect solver failures */
static const CCTK_REAL UTSQ_TOO_BIG =1.e20;    /* \tilde{u}^2 is assumed to be smaller
                                                  than this.  Used to detect solver
                                                  failures */

static const CCTK_REAL FAIL_VAL     =1.e30;    /* Generic value to which we set variables when a problem arises */

static const CCTK_REAL NUMEPSILON=(2.2204460492503131e-16);


/* some mnemonics */
/* for primitive variables */
static const int RHO    =0; 
static const int UU     =1;
static const int UTCON1 =2;
static const int UTCON2 =3;
static const int UTCON3 =4;
static const int BCON1  =5;
static const int BCON2  =6;
static const int BCON3  =7;

/* for conserved variables */
static const int QCOV0  =1;
static const int QCOV1  =2;
static const int QCOV2  =3;
static const int QCOV3  =4;

/********************************************************************************************/
// Function prototype declarations:
int Utoprim_2d(CCTK_REAL U[NPR], CCTK_REAL gcov[NDIM][NDIM], CCTK_REAL gcon[NDIM][NDIM], 
               CCTK_REAL gdet, CCTK_REAL prim[NPR], long &n_iter);

inline int harm_primitives_gammalaw_lowlevel(const int index,const int i,const int j,const int k,CCTK_REAL *X,CCTK_REAL *Y,CCTK_REAL *Z,
                                             CCTK_REAL *METRIC,CCTK_REAL *METRIC_PHYS,CCTK_REAL *METRIC_LAP_PSI4,
                                             CCTK_REAL *CONSERVS,CCTK_REAL *PRIMS,
                                             CCTK_REAL g4dn[NDIM][NDIM],CCTK_REAL g4up[NDIM][NDIM],
                                             output_stats &stats,eos_struct &eos);

inline int font_fix_general_gamma(CCTK_REAL &u_x, CCTK_REAL &u_y, CCTK_REAL &u_z,CCTK_REAL *CONSERVS,CCTK_REAL *PRIMS,CCTK_REAL *METRIC_PHYS,CCTK_REAL *METRIC_LAP_PSI4,eos_struct &eos);
inline int font_fix_gamma_equals2(CCTK_REAL &u_x, CCTK_REAL &u_y, CCTK_REAL &u_z,CCTK_REAL *CONSERVS,CCTK_REAL *PRIMS,CCTK_REAL *METRIC_PHYS,CCTK_REAL *METRIC_LAP_PSI4,eos_struct &eos);

void eigenvalues_3by3_real_sym_matrix(CCTK_REAL & lam1, CCTK_REAL & lam2, CCTK_REAL & lam3,
                                      CCTK_REAL M11, CCTK_REAL M12, CCTK_REAL M13, CCTK_REAL M22, CCTK_REAL M23, CCTK_REAL M33);

/********************************************************************************************/

#endif /* HARM_PRIMITIVES_HEADERS_H_ */



Overwriting ../src/harm_primitives_headers.h


<a id='code_validation'></a>

# Step 5: Code validation \[Back to [top](#toc)\]
$$\label{code_validation}$$

<a id='driver_conserv_to_prims_validation'></a>

## Step 5.a: `driver_conserv_to_prims.C` \[Back to [top](#toc)\]
$$\label{driver_conserv_to_prims_validation}$$

First we download the original `IllinoisGRMHD` source code and then compare it to the source code generated by this tutorial notebook.

In [20]:
# Verify if the code generated by this tutorial module
# matches the original IllinoisGRMHD source code

# First download the original IllinoisGRMHD source code
import urllib
from os import path

original_IGM_file_url  = "https://bitbucket.org/zach_etienne/wvuthorns/raw/2b402d7eaf8b177e5979cdfd3b73fd992dd2fa7e/IllinoisGRMHD/src/driver_conserv_to_prims.C"
original_IGM_file_name = "driver_conserv_to_prims-original.C"
original_IGM_file_path = IGM_src_dir_path+"/"+original_IGM_file_name
urllib.urlretrieve(original_IGM_file_url,original_IGM_file_path)

# Check if the downloaded file exists
# If it doesn't, error out
if not path.exists(original_IGM_file_path):
    print("Error, file "+original_IGM_file_path+" not found.")

# If it does, perform validation
else:
    Validation__driver_conserv_to_prims__C  = !diff $original_IGM_file_path $outfile_path__driver_conserv_to_prims__C

    if Validation__driver_conserv_to_prims__C == []:
        # If the validation passes, we do not need to store the original IGM source code file
        !rm $original_IGM_file_path
        print("Validation test for driver_conserv_to_prims.C: PASSED!")
    else:
        # If the validation fails, we keep the original IGM source code file
        print("Validation test for driver_conserv_to_prims.C: FAILED!")
        # We also print out the difference between the code generated
        # in this tutorial module and the original IGM source code
        print("Diff:")
        for diff_line in Validation__driver_conserv_to_prims__C:
            print(diff_line)

Validation test for driver_conserv_to_prims.C: FAILED!
Diff:
51a52,53
>   // Leo says: This is definitely one of the places we need to look at later.
>   //           Will leave this comment to emphasize the "FIXME" below.


<a id='harm_primitives_lowlevel_validation'></a>

## Step 5.b: `harm_primitives_lowlevel.C` \[Back to [top](#toc)\]
$$\label{harm_primitives_lowlevel_validation}$$

First we download the original `IllinoisGRMHD` source code and then compare it to the source code generated by this tutorial notebook.

In [21]:
# Verify if the code generated by this tutorial module
# matches the original IllinoisGRMHD source code

# First download the original IllinoisGRMHD source code
import urllib
from os import path

original_IGM_file_url  = "https://bitbucket.org/zach_etienne/wvuthorns/raw/2b402d7eaf8b177e5979cdfd3b73fd992dd2fa7e/IllinoisGRMHD/src/harm_primitives_lowlevel.C"
original_IGM_file_name = "harm_primitives_lowlevel-original.C"
original_IGM_file_path = IGM_src_dir_path+"/"+original_IGM_file_name
urllib.urlretrieve(original_IGM_file_url,original_IGM_file_path)

# Check if the downloaded file exists
# If it doesn't, error out
if not path.exists(original_IGM_file_path):
    print("Error, file "+original_IGM_file_path+" not found.")

# If it does, perform validation
else:
    Validation__harm_primitives_lowlevel__C  = !diff $original_IGM_file_path $outfile_path__harm_primitives_lowlevel__C

    if Validation__harm_primitives_lowlevel__C == []:
        # If the validation passes, we do not need to store the original IGM source code file
        !rm $original_IGM_file_path
        print("Validation test for harm_primitives_lowlevel.C: PASSED!")
    else:
        # If the validation fails, we keep the original IGM source code file
        print("Validation test for harm_primitives_lowlevel.C: FAILED!")
        # We also print out the difference between the code generated
        # in this tutorial module and the original IGM source code
        print("Diff:")
        for diff_line in Validation__harm_primitives_lowlevel__C:
            print(diff_line)

Validation test for harm_primitives_lowlevel.C: PASSED!


<a id='harm_primitives_headers_validation'></a>

## Step 5.c: `harm_primitives_headers.h` \[Back to [top](#toc)\]
$$\label{harm_primitives_headers_validation}$$

First we download the original `IllinoisGRMHD` source code and then compare it to the source code generated by this tutorial notebook.

In [22]:
# Verify if the code generated by this tutorial module
# matches the original IllinoisGRMHD source code

# First download the original IllinoisGRMHD source code
import urllib
from os import path

original_IGM_file_url  = "https://bitbucket.org/zach_etienne/wvuthorns/raw/2b402d7eaf8b177e5979cdfd3b73fd992dd2fa7e/IllinoisGRMHD/src/harm_primitives_headers.h"
original_IGM_file_name = "harm_primitives_headers-original.h"
original_IGM_file_path = IGM_src_dir_path+"/"+original_IGM_file_name
urllib.urlretrieve(original_IGM_file_url,original_IGM_file_path)

# Check if the downloaded file exists
# If it doesn't, error out
if not path.exists(original_IGM_file_path):
    print("Error, file "+original_IGM_file_path+" not found.")

# If it does, perform validation
else:
    Validation__harm_primitives_headers__h  = !diff $original_IGM_file_path $outfile_path__harm_primitives_headers__h

    if Validation__harm_primitives_headers__h == []:
        # If the validation passes, we do not need to store the original IGM source code file
        !rm $original_IGM_file_path
        print("Validation test for harm_primitives_headers.h: PASSED!")
    else:
        # If the validation fails, we keep the original IGM source code file
        print("Validation test for harm_primitives_headers.h: FAILED!")
        # We also print out the difference between the code generated
        # in this tutorial module and the original IGM source code
        print("Diff:")
        for diff_line in Validation__harm_primitives_headers__h:
            print(diff_line)


Validation test for harm_primitives_headers.h: PASSED!


<a id='latex_pdf_output'></a>

# Step 6: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-IllinoisGRMHD__driver_conserv_to_prims.pdf](Tutorial-IllinoisGRMHD__driver_conserv_to_prims.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means).

In [23]:
!jupyter nbconvert --to latex --template ../../latex_nrpy_style.tplx Tutorial-IllinoisGRMHD__driver_conserv_to_prims.ipynb
!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__driver_conserv_to_prims.tex
!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__driver_conserv_to_prims.tex
!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__driver_conserv_to_prims.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-IllinoisGRMHD__driver_conserv_to_prims.ipynb to latex
[NbConvertApp] Writing 93428 bytes to Tutorial-IllinoisGRMHD__driver_conserv_to_prims.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
